# Loading Data

In [1]:
csv_path = '../input/life-expectancy-who-pre-process/Life_expectancy_last_column.csv'

import pandas as pd
df = pd.read_csv(csv_path, index_col=0)
df.head()

,Year,Adult Mortality,Infant Deaths,Alcohol,Expenditure(%),Hepatitis B,Measles,BMI,Under-five Deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,Thinness 1-19 Years,Thinness 5-9 Years,ICOR,Schooling,Life Expectancy
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


# Considerations

In [2]:
Similar_event_count = 100
sensitivity_consideration = 1
error_sensitivity_consideration = 1

# Error-Signal Ratio
ES_ratio = "Yes"

# Data Arranging and Normalizing

In [3]:
input_=[]
output_=[]
i_val=[]
o_val=[]
i_test=[]
o_test=[]
i_all =[]
o_all=[]
iter1=0

for row in df.iloc:
    i_all.append((row[0:len(row)-1]).astype(float))
    o_all.append(row[-1])
    if iter1%10 >1:
        input_.append((row[0:len(row)-1]).astype(float))
        output_.append(row[-1])
    if iter1%10 ==0:
        i_val.append((row[0:len(row)-1]).astype(float))
        o_val.append(row[-1])
    if iter1%10 ==1:
        i_test.append((row[0:len(row)-1]).astype(float))
        o_test.append(row[-1])
    iter1 = iter1 + 1
  

    
num_input = len(row)-1;
print(num_input)

####################################################
#
# This code is written with the help of the demo:
# https://medium.com/@benjamin.phillips22/simple-regression-with-neural-networks-in-pytorch-313f06910379
#
####################################################
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import imageio

torch.manual_seed(1)    # reproducible

x = torch.tensor(input_).float()  
y = torch.tensor(output_).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)


i_val = torch.tensor(i_val).float()  
o_val = torch.tensor(o_val).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)



max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

    #Normalizing
i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y

19
tensor(89.) tensor(44.3000) tensor([2.0150e+03, 7.2300e+02, 1.6000e+03, 1.7870e+01, 1.8961e+04, 9.9000e+01,
        1.3144e+05, 7.6700e+01, 2.1000e+03, 9.9000e+01, 1.4390e+01, 9.9000e+01,
        5.0600e+01, 1.1576e+05, 1.2939e+09, 2.7200e+01, 2.8200e+01, 9.3600e-01,
        2.0700e+01]) tensor([2.0000e+03, 1.0000e+00, 0.0000e+00, 1.0000e-02, 0.0000e+00, 2.0000e+00,
        0.0000e+00, 2.0000e+00, 0.0000e+00, 3.0000e+00, 7.4000e-01, 4.0000e+00,
        1.0000e-01, 1.6813e+00, 3.4000e+01, 1.0000e-01, 1.0000e-01, 0.0000e+00,
        4.5000e+00])
tensor([1.5000e+01, 7.2200e+02, 1.6000e+03, 1.7860e+01, 1.8961e+04, 9.7000e+01,
        1.3144e+05, 7.4700e+01, 2.1000e+03, 9.6000e+01, 1.3650e+01, 9.5000e+01,
        5.0500e+01, 1.1576e+05, 1.2939e+09, 2.7100e+01, 2.8100e+01, 9.3600e-01,
        1.6200e+01]) tensor(44.7000)


# Prediction NN Training

In [4]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

# use the same net as before      
net = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5 #High initial values
minimum_val_loss = 1e5
EPOCH = 600

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net(
  (hidden): Linear(in_features=19, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/600], Loss: 0.0182, Minimum Loss 0.018225, Val Loss 0.019159  
Epoch [400/600], Loss: 0.0101, Minimum Loss 0.010058, Val Loss 0.010342  
Epoch [600/600], Loss: 0.0072, Minimum Loss 0.007196, Val Loss 0.007345  


# Computing Test Loss

In [5]:
 
i_test, o_test = Variable(torch.tensor(i_test)).float(), Variable(torch.tensor(o_test).float())
new_shape = (len(o_test), 1)
o_test = o_test.view(new_shape)

    #Normalizing
i_test = (i_test - min_x.values)/range_x
o_test = (o_test - min_y)/range_y

prediction = net_opt_val(i_test)
loss_test = loss_func(prediction, o_test)

print("Normalized Test Loss",loss_test.detach().numpy())

loss_test = loss_test*range_y*range_y # As the loss function returns MSE

print("Test Loss",loss_test.detach().numpy())

Normalized Test Loss 0.0071752337
Test Loss 14.336763


# NN for Predicting Heteroscedastic Absolute Error of Previous NN 

In [6]:
y_e = torch.zeros(len(x), dtype=torch.float) # initializing a variable of that size

for iter1 in range(len(x)):
    current_input = x[iter1,:];
    current_output = net_opt_val(current_input);
    y_e[iter1] = torch.abs(current_output - y[iter1]) 
        
x, y_e = Variable(x), Variable(y_e)
net_e = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net_e)  # net architecture

# Error to Signal ratio
ratio_e_s = 0.5 #default
if ES_ratio == "Yes":
    ratio_e_s = torch.var(y_e)/torch.var(y)

    
print('Error to Signal ratio:', ratio_e_s)

optimizer = torch.optim.Adam(net_e.parameters(), lr=0.05)

minimum_train_loss = 1e5 #High initial values
minimum_val_loss = 1e5
EPOCH = 600


for epoch in range(EPOCH):
  
    prediction = net_e(x)     # input x and predict based on x

    loss = loss_func(prediction, y_e)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net_e
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val_e = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))
    



Net(
  (hidden): Linear(in_features=19, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Error to Signal ratio: tensor(0.0749)


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1319])) that is different to the input size (torch.Size([1319, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [200/600], Loss: 0.0029, Minimum Loss 0.002879, Val Loss 0.275328  
Epoch [400/600], Loss: 0.0029, Minimum Loss 0.002879, Val Loss 0.275328  
Epoch [600/600], Loss: 0.0029, Minimum Loss 0.002879, Val Loss 0.275328  


# Minimim deviation based similarity finding for Uncertainty Bounds

   Aim: To achieve all indexes of similar samples

   Steps: 

1. Itertion over all Input combinations.

2. Find all deviation by indexes 
   (sensitivity can be considered during the consideration of deviation)

3. Save top 100 similar events


In [7]:
import tensorflow as tf
import pickle as pkl
import time


x = torch.tensor(i_all).float()  
y = torch.tensor(o_all).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y
x, y = Variable(x), Variable(y)


max_dev_index = np.zeros(shape=(len(x),2))
Similarity = np.zeros(shape=(len(x),Similar_event_count+1))
sensitivity = torch.ones(len(max_x.values), dtype=torch.float64) # just initializing a variable of that size
error_sensitivity = torch.ones(len(max_x.values), dtype=torch.float64)
iter_debug = 0
since = time.time()

for iter1 in range(len(x)):
    current_input = x[iter1,:];
    
    # While considering only Sensitivity
    if sensitivity_consideration == 1 and error_sensitivity_consideration ==0:
        current_output = net_opt_val(current_input)
        
       # Determining Sensitivity near input[iter1]
        for iter2 in range(len(current_input)): 
            current_input[iter2] = current_input[iter2] + range_x[iter2]/1e4
            
            sensitivity[iter2] = torch.abs(current_output - net_opt_val(current_input))
            
            current_input[iter2] = current_input[iter2] - range_x[iter2]/1e4
        
        sensitivity = sensitivity/torch.max(sensitivity)
        #The highest sensitivity is considered one
    
    if sensitivity_consideration == 1 and error_sensitivity_consideration ==1:
        current_output = net_opt_val(current_input)
        current_error = net_opt_val_e(current_input)
        # ratio, ratio_e_s= 0.5 while giving equal concentration to error and signal
        
       # Determining Sensitivity and Error Sensitivity near input[iter1]
        for iter2 in range(len(current_input)): 
            current_input[iter2] = current_input[iter2] + range_x[iter2]/1e4
            
            sensitivity[iter2] = torch.abs(current_output - net_opt_val(current_input))
            error_sensitivity[iter2] = torch.abs(current_error - net_opt_val_e(current_input))
            
            current_input[iter2] = current_input[iter2] - range_x[iter2]/1e4
        
        sensitivity = sensitivity/torch.max(sensitivity)*(1-ratio_e_s)
        error_sensitivity = error_sensitivity/torch.max(error_sensitivity)* ratio_e_s
        sensitivity = sensitivity + error_sensitivity
        
        #The highest sensitivity is considered half for each, then adding
        
    
    for iter2 in range(len(x)):
        max_dev_index[iter2][0] = torch.max(sensitivity*torch.abs(current_input-x[iter2,:])/range_x).detach().numpy()
        max_dev_index[iter2][1] = iter2
        
    sort_dev_index = max_dev_index[np.argsort(max_dev_index[:, 0])]
    Similarity_threshold = sort_dev_index[Similar_event_count][0]
    matched_indexes = sort_dev_index[0:Similar_event_count,1]
    
    Similarity[iter1,0] = Similarity_threshold
    Similarity[iter1,1:(Similar_event_count+1)] = matched_indexes
    
    if iter1%500==100:   
        print(iter1/len(x)*100, '%' '  Complete')
        time_elapsed = time.time() - since
        # Observing progres in Command Window
        # May Take about hours to find all similar events for entire training data

print('100% Complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

fileName = './' + 'Similarity'
fileObject = open(fileName, 'wb')

pkl.dump(Similarity, fileObject)
fileObject.close()



6.064281382656155 %  Complete
36.38568829593693 %  Complete
66.7070952092177 %  Complete
97.02850212249848 %  Complete
100% Complete in 2m 34s
